# Setup

In [ ]:
# Update the settings to point to the datasets and runs directories
from ultralytics import settings
from ct_detector import DATASETS_DIR, ROOT_DIR
import os

settings.update({'datasets_dir': DATASETS_DIR, 'runs_dir': os.path.join(ROOT_DIR, 'runs')})

# Single model predictions

In [ ]:
# Import the necessary libraries and our custom predictor
from ultralytics import YOLO
from ct_detector.model.predict import CtPredictor
from ct_detector.model import MODELS, DATASETS, DATASETS_DIR
import os

# Load a model
model_path = MODELS['eie_t_1_yolov8m']  # or your custom model: 'path/to/best.pt'

# Get a dataset
data_path = os.path.join(DATASETS_DIR, "1", "val.txt")  # or your dataset .yaml or folder of images

# Prepare the predictor and set up its parameters
analysis_pred = CtPredictor(
    overrides={
        "conf": 0.3,      # confidence threshold
        "save": False,    # don't save annotated images
        "save_txt": False, # no label files
    },
    handle_existing_labels="skip",  # not used since we aren't saving
    labels_dir="",
)

# Load the model and assign the predictor
model = YOLO(model_path)
model.predictor = analysis_pred

# Now run predictions on a folder or an image. Stream =True is used to get results as they are processed in a form of a generator that we can iterate over as shown below.
results = model.predict(data_path, stream=True)

# results_single_analysis is a generator of Results. Let's just show five
counter = 0
for r in results:
    counter += 1
    if counter < 5:
        print(f"Image: {r.path}, #boxes={len(r.boxes)}")

In [ ]:
# Import the necessary libraries and our custom predictor
from ultralytics import YOLO
from ct_detector.model.predict import CtPredictor
from ct_detector.model import MODELS, DATASETS, DATASETS_DIR
from IPython.display import display
from PIL import Image
import os

# Load a model
model_path = MODELS['eie_t_1_yolov8m']  # or your custom model: 'path/to/best.pt'

# Get a dataset
data_path = os.path.join(DATASETS_DIR, "1", "val.txt")  # or your dataset .yaml or folder of images

# We can define a callback function to be called at different stages of the prediction process. It always accepts the predictor as its argument.
def demo_callback(predictor):
    """
    Example post-inference callback that plots the results.
    """
    for r in predictor.results:
        display(Image.fromarray(r.plot()).resize((400, 400)))

# You can specify callbacks for each stage of the prediction process
from ultralytics.utils.callbacks import default_callbacks
predictor_callbacks = default_callbacks.copy()
predictor_callbacks['on_predict_batch_end'] = [demo_callback]

# Prepare the predictor and set up its parameters including the callbacks
analysis_pred = CtPredictor(
    overrides={
        "conf": 0.3,      # confidence threshold
        "save": False,    # don't save annotated images
        "save_txt": False, # no label files
        "verbose": False # no verbose output
    },
    _callbacks=predictor_callbacks,
    handle_existing_labels="skip",  # not used since we aren't saving
    labels_dir="",
)

# Load the model and assign the predictor
model = YOLO(model_path)
model.predictor = analysis_pred

# Alternatively we assign the callbacks to the predictor after loading the model
# model.predictor.callbacks = predictor_callbacks

# Now run predictions on a folder or an image. Stream =False is used to get results at once in a form of a list of Results. They are stored in memory and can be accessed via the predictor.results attribute. However, this is not recommended for large datasets.
results = model.predict(data_path, stream=False)

# results is a list of Results. Let's just show five
for r in results[:5]:
    print(f"Image: {os.path.basename(r.path)}, #boxes={len(r.boxes)}")

In [ ]:
# Import the necessary libraries and our custom predictor
from ultralytics import YOLO
from ct_detector.model.predict import CtPredictor
from ct_detector.model import MODELS, DATASETS, DATASETS_DIR
import os

# Load a model
model_path = MODELS['eie_t_1_yolov8m']  # or your custom model: 'path/to/best.pt'

# Get a dataset
data_path = os.path.join(DATASETS_DIR, "1", "val.txt")  # or your dataset .yaml or folder of images

# Prepare the predictor and set up its parameters, this time allowing auto-annotation
auto_annot_pred = CtPredictor(
    overrides={
        "conf": 0.25,
        "save": True,           # save annotated images
        "save_txt": True       # save .txt label files
    },
    handle_existing_labels="rename",   # rename existing .txt
    labels_dir="custom_labels",        # store them in custom_labels/ folder
)

# Load the model and assign the predictor
model = YOLO(model_path)
model.predictor = auto_annot_pred

# Now run predictions on a folder or an image
results = model.predict(data_path, stream=False)

# Multi-model predictions

In [ ]:
from ct_detector.model.ensemble import CtEnsembler
from ct_detector.model import MODELS, DATASETS, DATASETS_DIR
import os
from IPython.display import display
from PIL import Image

# Get a dataset
data_path = os.path.join(DATASETS_DIR, "1", "val.txt")  # or your dataset .yaml or folder of images

def example_callback(merged_result, frame_idx):
    display(Image.fromarray(merged_result.plot()).resize((400, 400)))

# 1) Create the ensemble
ensembler = CtEnsembler(
    model_paths=[MODELS['eie_t_1_yolov8m'], MODELS['eie_t_1_yolov9m']],
    predictor_overrides={"conf": 0.3, "save_txt": False, "verbose": False},  # e.g. no partial labels
)

# 2) Run predictions on a directory of images
gen = ensembler.predict(
    source=data_path,
    nms_iou_thres=0.5,
    nms_conf_thres=0.3,
    class_agnostic=False,
    class_merge_map={0:0, 1:1, 2:0, 3:2},  # Merge classes 0 and 2
    frame_callback=example_callback
)

# 3) For each merged frame, do more logic if you want
for idx, merged_frame in enumerate(gen):
    # Possibly do final .txt saving, or display bounding boxes
    # merged_frame.save_txt(f"ensemble_labels/frame_{idx}.txt")
    pass